# Training Deep Neural Networks

This notebook is heavily inspired by Andre Guernon work, that can be found here: https://github.com/ageron/handson-ml/blob/master/11_deep_learning.ipynb

To tackle some very complex problems, you may have to design and train a very deep network (> 10 layers), with complex connectivity. This could generate a variety of issues
* vanishing / exploding gradients
* too little data or not enough labels for our data
* training could be very time-consuming
* there would be a high risk of overfitting your data

Let's see, one by one, how we can sort out these issue

## Setup

In [ ]:
# Python ≥ 3.8 is required
import sys
assert sys.version_info >= (3, 8)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "1.0"

# Common imports
import numpy as np
import pandas as pd
import os

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Ignore useless warnings (see SciPy issue #5998)
import warnings
warnings.filterwarnings(action="ignore", message="^internal gelsd")

In [ ]:
import tensorflow as tf
from tensorflow import keras
warnings.filterwarnings(action="ignore", message="^NUMA")

## 1. Vanishing/Exploding Gradients

The _vanishing gradient_ problem happens when training artificial neural networks with gradient-based learning methods and backpropagation. In such methods, each of the neural network's weights receives an update proportional to the partial derivative of the error function with respect to the current weight in each iteration of training. The problem is that in some cases, the gradient will be vanishingly small, effectively preventing the weight from changing its value. In the worst case, this may completely stop the neural network from further training. This will be especially true as the error back-propagates to the lower layers.

In other cases, however, it can happen exactly the opposite: the gradients get bigger and bigger, with consequently very large weight updates, and a divergence of the algorithm. This is called the _exploding gradients_ problem.

Some of the causes of unstable gradients were found by Glorot and Bengio in a paper published in 2010. They found that the sigmoid activation function together with a popular weight initialization technique used at that time caused a huge variance in each layer output, and this variance tended to increase layer by layer.

When inputs have a large absolute value, the sigmoid function saturates (to 0 in input negative, to 1 if positive), and the derivative tends to zero. Hence, there is no gradient to backpropagate through the network.

For the signal to flow properly across the network both forwards and backwards, the variance of the outputs must be equal to the variance of the inputs. In order to achieve this the weights must be initialized according to _Glorot Initialization_ rule:

$$ Normal(0, \frac{1}{fan_{avg}}) $$

or

$$ Uniform(-\sqrt{\frac{3}{fan_{avg}}}, \sqrt{\frac{3}{fan_{avg}}})$$

where

$$fan_{avg} = \frac{fan_{in} + fan_{out}}{2}$$

and 

$$fan_{in}$$ is the number of inputs and $$fan_{out}$$ is the number of neurons (i.e. of outputs) for each layer

Other types of initialization are suggested for other activation functions:

|Initialization |Activation functions         |$\sigma^2$(Normal)
|---------------|-----------------------------|-------------------
|Glorot         |None, sigmoid, tanh, softmax | $1/fan_{avg}$
|He             |ReLU and    its variants     | $2/fan_{in}$
|LeCun          |SELU                         | $1/fan_{in}$

Keras adopts Glorot initialization by default. Let's try and set the iniziatlization to "He" for one layer.

In [ ]:
# He initialization with Normal distribution
keras.layers.Dense(
    10, activation='relu', 
    kernel_initializer=keras.initializers.he_normal()
)
# He initialization with Uniform distribution
he_avg_init = keras.initializers.VarianceScaling(
    scale=2., mode='fan_avg', distribution='uniform'
)
keras.layers.Dense(10, activation='sigmoid', kernel_initializer=he_avg_init)

Glorot and Bengio found out that ReLu works much better than sigmoid and tanh. However ReLu has its problems: (1) during training many neurons "die", i.e. start outputting only zeros. This happens especially with a very high learning rate. This happens, because the ReLu outputs zero for all negative values.

To solve this issue a few variations of ReLU have been proposed:
* Leaky ReLu
* Randomized Leaky ReLu (RReLU)
* Parametric Leaky ReLu (PReLU)
* Exponential Linear Unit (ELU) [Djork-Arné Clevert et al. (2015)]
* Scaled ELU (SELU) [Günter Klambauer et al. (2017)]

If a neural network is built exclusively of a stack of dense layers, and all the hidden layers use the SELU activation function, then the network will self-normalize. This means that the output of each layer will tend to preserve a mean of 0 and standard deviation of 1 during training, thus solving the issue of the vanishing/exploding gradients. Hence, the SELU activation function often significantly outperforms other activation functions for such neural nets (especially deep ones). There are, however, some conditions that are needed for self-normalization to happen:

* Inputs must be standardized, with mean 0 and std 1
* hidden layer’s weights must be initialized with LeCun normal initialization. 
* the network’s architecture must be sequential. This won't work with recurrent networks, or other more complicated architectures
* in theory all layers must be dense, however SELU improves performance in convolutional neural networks as well.

As an overall rule for the choice of activation functions: SELU > ELU > leaky ReLU (and its variants) > ReLU > tanh > logistic. However, if the network architecture prevents self-normalization, ELU might perform better than SELU.If speed is a priority many libraries and hardware accelerators provide ReLU-dadicated optimizations. Hence, if speed is priority, then ReLU is the choice.

In [ ]:
layer = keras.layers.Dense(10, activation='selu', kernel_initializer='lecun_normal')

### 1.2 Batch Normalization

Even though He initialization with SELU removes the issue of exploding/vanishing gradients at the beginning of the training it does not guarantee that they won't show again later on during the training phase.
A technique called _Batch Normalization_ (proposed by Ioffe and Szegedy, 2015: https://arxiv.org/abs/1502.03167) can solve this problem. 

"Batch normalization is achieved through a normalization step that fixes the means and variances of each layer's (hidden layers included) inputs. Ideally, the normalization would be conducted over the entire training set, but to use this step jointly with stochastic optimization methods, it is impractical to use the global information. Thus, normalization is restrained to each mini-batch in the training process." (Wikipedia)

To zero-center and normalize the inputs, the algorithm needs to estimate the mean value and standard deviation of each input. It does so by evaluating the mean and standard deviation of the input over the current mini-batch of data.

The problem comes at test time, when we should evaluate one item at the time and in principle we cannot compute mean and std over mini-batches. Keras's implementation of Batch Normalization estimates these test-time statistics during training. It does so using the moving average (https://www.investopedia.com/terms/m/movingaverage.asp) of the layer’s input means and standard deviations. 

In [ ]:
# Example of Batch Normalization in Keras
n_net = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(300, activation="elu", kernel_initializer="he_normal"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(100, activation="elu", kernel_initializer="he_normal"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(10, activation="softmax")
])

In [ ]:
(4*784 + 4*300 + 4*100)/2

In [ ]:
n_net.summary()

Each Batch Normalisation layer adds four parameters per input: $\gamma$, $\beta$, $\mu$, and $\sigma$. The last two parameters are the moving averages and are not trainable with Keras, while the other two are trainable by backpropagation.

Ioffe and Szegedy advocated in favour of adding the Batch Normalization layers before the activation functions, not after (as we did above). If you want to do as they suggested, you have to remove the activation function from the dense layer, and add a separate activation layer after the BN step 

In [ ]:
n_net = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(300, kernel_initializer='he_normal', 
                       use_bias=False),
    keras.layers.BatchNormalization(),
    keras.layers.Activation('elu'),
    keras.layers.Dense(100, kernel_initializer='he_normal', 
                       use_bias=False),
    keras.layers.BatchNormalization(),
    keras.layers.Activation('elu'),
    keras.layers.Dense(10, activation='softmax')
])

### 1.3 Gradient Clipping

Another popular approach to cope with the exploding gradients problem is to clip the gradients during backpropagation so that they never exceed some threshold. This technique is often used in Recurrent Neural Networks, where Batch Normalisation is difficult to perform.

In Keras, implementing Gradient Clipping is just a matter of setting the clipvalue or clipnorm argument when creating an optimizer, 

In [ ]:
optimizer = keras.optimizers.SGD(clipvalue=1.0)
n_net.compile(loss="mse", optimizer=optimizer)

## 2. Reusing Pre-trained Layers

Usually, it is not convenient to train a very large Deep Neural Network form scratch. You should approach the problem trying to identify an existing NN that accomplishes a similar task to the one you are trying to solve, then reuse the lower layers of this network. 
This approach is called transfer learning.

In [ ]:
net_A = keras.models.Sequential()
net_A.add(keras.layers.Flatten(input_shape=[28, 28]))
for n_hidden in (300, 100, 50, 50, 50):
    net_A.add(keras.layers.Dense(n_hidden, activation="selu"))
net_A.add(keras.layers.Dense(8, activation="softmax"))
# No code for this bit for now
net_A.compile(loss="sparse_categorical_crossentropy",
                optimizer=keras.optimizers.SGD(lr=1e-3),
                metrics=["accuracy"])
# now you should train your network
# history = net_A.fit()

In [ ]:
# save the network
net_A.save("my_net_A.h5")

In [ ]:
net_A = keras.models.load_model("my_net_A.h5")
# clone the network (so that we do not modify the original)
net_A_clone = keras.models.clone_model(net_A)
# we need to cpy the weights after cloning
net_A_clone.set_weights(net_A.get_weights())
net_B_on_A = keras.models.Sequential(net_A_clone.layers[:-1])
net_B_on_A.add(keras.layers.Dense(1, activation="sigmoid"))

In [ ]:
# freeze the lower layers' weights so that they won't be changed during the 
# first epochs of training
for layer in net_B_on_A.layers[:-1]:
    layer.trainable = False

net_B_on_A.compile(loss="binary_crossentropy",
                   optimizer=keras.optimizers.SGD(lr=1e-3),
                   metrics=["accuracy"])

## now you should train your model for a few epochs
## history = history = model_B_on_A.fit(epochs=4)

## unfreeze the lower layers
for layer in net_B_on_A.layers[:-1]:
    layer.trainable = True

# you need to recompile your network after unfreezing
net_B_on_A.compile(loss="binary_crossentropy",
                   optimizer=keras.optimizers.SGD(lr=1e-3),
                   metrics=["accuracy"])

## now you can train your model for the remaining epochs
## history = history = model_B_on_A.fit(epochs=36)

### 2.1 Unsupervised Pretraining



Oftentimes it is cheap to collect unlabeled training examples, but expensive to label them. If you can get plenty of unlabeled training data, you can try to first use it to train some unsupervised network, such as an autoencoder or a generative adversarial network. Afterwards, you an reuse the lower layers of the unsupervised model you've just trained, add the output layer(s) for your task on top of them, and adjust the final network using supervised learning.

Geoffrey Hinton adopted this approach in 2006 and wit led to the revival of ANN and the success of Deep Learning. 

Other approaches involve _self-supervised learning_ which is when you automatically generate the labels from input data itself, then you train a model on the resulting “labeled” dataset using supervised learning techniques. This approach doesn't require human labeling/supervision, so it is most often considered as a form of unsupervised learning. For instance, for natural language processing (NLP) applications, you can get massive corpora of text documents and automatically generate labels from it.

## 3. Faster Optimizers

You can find an overview on optimizers here: https://towardsdatascience.com/full-review-on-optimizing-neural-network-training-with-optimizer-9c1acc4dbe78

### Reminder: Gradient Descent

$$ \theta^{(t+1)} = \theta^{(t)} - \eta \nabla_{\theta} J(\theta) $$

### Momentum Optimization

Instead of using only the gradient of the current step to guide the search, momentum optimisation also accumulates the gradient of the past steps to determine the direction to go. 

$$ m^{(t+1)} = \beta m^{(t)} - \eta \nabla_{\theta} J(\theta) $$
$$ \theta^{(t+1)} = \theta^{(t)} + m $$

In [ ]:
optimizer = keras.optimizers.SGD(lr=0.001, momentum=0.9)

### Nesterov Accelerated Gradient

$$ m^{(t+1)} = \theta m^{(t)} - \eta \nabla_{\theta} J(\theta + \beta m) $$
$$ \theta^{(t+1)} = \theta^{(t)} + m $$

A variant of momentum optimisation. Nesterov update ends up slightly closer to the optimum. After a while, these small improvements add up and NAG ends up being significantly faster than regular momentum optimization. 

In [ ]:
optimizer = keras.optimizers.SGD(
    lr=0.001, momentum=0.9, nesterov=True
)

### Adagrad 

This algorithm decays the learning rate, but this happens faster for steep dimensions than for dimensions with less pronounced slopes. It does this by accumulating the squares of gradients and scaling the learning rate with respect to the accumulated squares. This is called an adaptive learning rate. Not suitable for Deep Neural Networks

### RMSProp

AdaGrad runs the risk of slowing down a bit too fast and never converging to the global optimum. The RMSProp algorithm fixes this by accumulating only the gradients from the most recent iterations (as opposed to all the gradients since the beginning of training). It does so by using exponential decay by a decay rate $\rho$

RMSprop uses a (moving) average of squared gradients to normalize the gradient. This normalization balances the step size  (momentum),  decreasing the step for large gradients to avoid exploding, and increasing the step for small gradients to avoid vanishing. 

In [ ]:
optimizer = keras.optimizers.RMSprop(
    lr=0.001,
    rho=0.9 # decay rate
)

### Adam

The *adaptive moment estimation*, combines the ideas of momentum optimization and RMSProp. It keeps track of an exponentially decaying average of past gradients ($\beta_1$ decay rate) and  of an exponentially decaying average of past squared gradients ($\beta_2$ decay rate).
Being and adaptive learning rate algorithm, you need less tuning of the learning rate, making it more flexible than standard gradient descent.
There are two more variants of Adam: AdaMAX (scales down the parameter updates of $\beta_2$ by the $l_{\inf}$ norm rather than the $l_{2}$ norm) and Nadam (Adam + Nesterov)

In [ ]:
optimizer = keras.optimizers.Adam(
    lr=0.001, beta_1=0.9, beta_2=0.999
)

### Summary of Optimisers

| Class                             | Convergence speed | Convergence quality   |
|-----------------------------------|-------------------|-----------------------|
|`SGD`                              |     Bad           |         Good          |
|`SGD(momentum=...)`                |     Average       |         Good          |
|`SGD(momentum=..., nesterov=True)` |     Average       |         Good          |
|`Adagrad`                          |     Good          | Bad (stops too early) |
|`RMSprop`                          |     Good          |     Average/Good      |
|`Adam`                             |     Good          |     Average/Good      |
|`AdaMax`                           |     Good          |     Average/Good      |
|`Nadam`                            |     Good          |     Average/Good      |

## 3. Learning Rate Scheduling

The idea here is to start with a large learning rate and decrease it as the number of iterations over the training set increases.

### Power LR scheduling

$$\eta[t] = \frac{\eta_{0}}{(1+t/s)^c}$$

In [ ]:
optimizer = keras.optimizers.SGD(
    lr=0.01,
    decay=1e-4 # decay = 1/s, c =1 for keras
) 

### Exponential LR scheduling

$$\eta[t] = \eta_{0} 0.1^{t/s}$$

In [ ]:
# define a function inside a function to pass eta0 and s
# This pattern is called factory method
def exponential_decay(lr_0, s):
    def exponential_decay_fn(epoch):
        return lr_0 * 0.1**(epoch / s)
    return exponential_decay_fn

# create your own decay function from the factory method
exponential_decay_fn = exponential_decay(
    lr_0=0.01, s=20
)

# create a LR scheduler callback passing your decay function
lr_scheduler = keras.callbacks.LearningRateScheduler(
    exponential_decay_fn
)

In [ ]:
# NB: this cell is not going to work as training set is empty.
X_train_scaled = []
y_train = []
history = n_net.fit(
    X_train_scaled,
    y_train,
    callbacks=[lr_scheduler] # pass the scheduler to the callbacks of .fit()
)

## 4. Regularization

### 4.1 ℓ1 and ℓ2 Regularization

Just like we did for Linear Regression in Week 3, we can use ℓ2 (i.e. "Lasso") regularization  to constrain a neural network’s connection weights, and/or ℓ1 (i.e "Ridge") regularization to achieve a sparse model (with many weights equal to 0). In the cell below we apply ℓ2 regularization to a Keras layer’s connection weights, using a regularization factor of 0.02:

In [ ]:
layer = keras.layers.Dense(
    100,
    activation="elu",
    kernel_initializer="he_normal",
    kernel_regularizer=keras.regularizers.l2(0.01)
)

### 4.2 Dropout

Dropout has become one of the most popular regularization techniques for deep neural networks. Originally proposed by Geoffrey Hinton in 2012 it was later exposed in more detail in a paper by Nitish Srivastava et al. (2014). It is a very successful technique: state-of-the-art neural networks get a 1–2% accuracy improvement just by adding dropout. This means that when a network already has 95% accuracy, getting a 2% accuracy increase means reducing the error by 40% (from 5% to about 3%).

The algorithm works like this: at each training step, each neuron (including input neurons, but excluding output neurons) has a probability p of being “dropped out” temporarily. This means that it will be completely ignored during the current training step, but it may be active during the following step. The hyperparameter _p_ is named dropout rate, and it usually falls between 10% and 50%: closer to 20–30% in recurrent neural nets (we will see some of these for NLP), and closer to 40–50% in convolutional neural networks (we will see these for Computer Vision mainly). After training, nthere is no more dropout.

In practice, dropout is only applied to the neurons in the top 1-3 layers (still excluding the output layer). See the example below of a fully connected Network with dropout rate of 20%. As you see all you have to do is to add a `Dropout` layer:

In [ ]:
n_net = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dropout(rate=0.2),
    keras.layers.Dense(
        300, activation="elu", kernel_initializer="he_normal"
    ),
    keras.layers.Dropout(rate=0.2),
    keras.layers.Dense(
        100, activation="elu", kernel_initializer="he_normal"
    ),
    keras.layers.Dropout(rate=0.2),
    keras.layers.Dense(10, activation="softmax")
])